In [2]:
from kubernetes import client, config,utils

# Kubernete Api Client


In [5]:
config.load_kube_config()
k8s_client = client.ApiClient() 
yaml_file = './vcjob.yaml'
utils.create_from_yaml(k8s_client,yaml_file,verbose=True)

AttributeError: module 'kubernetes.client' has no attribute 'BatchVolcanoShV1alpha1Api'

# Kubernetes CRD Client

In [8]:
volcano_job_body = {
    "apiVersion": "batch.volcano.sh/v1alpha1",
    "kind": "Job",
    "metadata": {
        "name": "test-job"
    },
    "spec": {
        "schedulerName": "volcano",
        "priorityClassName": "h-priority",
        "policies": [
            {
                "event": "PodEvicted",
                "action": "RestartJob"
            }
        ],
        "maxRetry": 5,
        "tasks": [
            {
                "name": "default-nginx",
                "template": {
                    "metadata": {
                        "name": "web"
                    },
                    "spec": {
                        "containers": [
                            {
                                "image": "nginx",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "nginx",
                                "resources": {
                                    "requests": {
                                        "cpu": "1"
                                    }
                                }
                            }
                        ]
                    }
                }
            }
        ]
    }
}

In [10]:
configuration = client.Configuration()
api = client.CustomObjectsApi()
api.create_namespaced_custom_object(
    group="batch.volcano.sh",
    version="v1alpha1",
    namespace="default",
    plural="jobs",
    body=volcano_job_body
)

print("Resource Created")
    

Resource Created
